In [1]:
import numpy as np
from scipy.optimize import least_squares
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
#
# if pade.py is not in the current directory, set this path:
#
import sys
sys.path.append('../Python_libs')
import rac_pade_functions as racx

In [2]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#  files in the current directory do not need the path name
#
#df = pd.read_csv("/home/thomas/Python/StabPlots/Stab_data/1D_a0.2_b0_c0.14/crossing_1.dat", delim_whitespace=True)
df = pd.read_csv("sb_rac.csv")
#df = pd.read_csv("crossing_1.dat", delim_whitespace=True)
plt.cla()
plt.plot(df.l.values, df.E1.values, 'o-')
plt.plot(df.l.values, df.E2.values, 'o-')
plt.plot(df.l.values, df.E3.values, 'o-')
plt.show()
df[:5]

,l,E1,E2,E3,E4
0,0.0,0.469219,1.365173,1.938043,3.271038
1,0.2,0.457846,1.314401,1.844238,3.242423
2,0.4,0.446073,1.251127,1.760926,3.215267
3,0.6,0.433754,1.173608,1.690292,3.189317
4,0.8,0.420648,1.082215,1.632448,3.164365


In [3]:
i_neg = np.argmin(abs(df.E1.values))
if df.E1[i_neg] > 0:
    i_neg += 1
ls = df.l.values[i_neg:]
print('N=',len(ls))
Es = df.E1.values[i_neg:]
plt.cla()
plt.plot(df.l.values, df.E1.values, 'b-')
plt.plot(df.l.values, df.E2.values, 'b-')
plt.plot(df.l.values, df.E3.values, 'b-')
plt.plot(ls, Es, 'o-', color="orange")
plt.show()

N= 89


In [4]:
k2s = -Es
ks  = np.sqrt(k2s)

In [5]:
#
#  this will go into a module at some point
#
def chi2_gen(params, ks, k2s, lbs, pade):
    """
    fit l = pade(k; params) to the data in ks, lbs using the weights sigma 
    """
    diffs = pade(ks, k2s, params) - lbs
    return np.sum(np.square(diffs)) / len(ks)

def chi2_gen_num_jac(params, ks, k2s, lbs, pade, step=1e-3, tiny=1e-6):
    """
    calc chi2 and the numeric gradient
    just for testing
    """
    n_kappa = len(ks)
    n_para = len(params)
    p0 = list(params)
    diffs = pade(ks, k2s, params) - lbs
    chi2 = np.sum(np.square(diffs)) / n_kappa
    
    dchi2 = np.zeros(n_para)
    for ip in range(n_para):
        h = params[ip]*step + tiny
        pm = np.array(p0[:ip] + [p0[ip]-h] + p0[ip+1:])
        pp = np.array(p0[:ip] + [p0[ip]+h] + p0[ip+1:])
        chi2_m = chi2_gen(pm, ks, k2s, lbs, pade)
        chi2_p = chi2_gen(pp, ks, k2s, lbs, pade)
        dchi2[ip] = (chi2_p - chi2_m)/(2*h)
    return chi2, dchi2
    
def pade_31(k, ksq, params):
    """ 
    Pade [3,1]  
    """
    l = params[0]
    a = params[1]
    b = params[2]
    d = params[3]
    a4b2=a*a*a*a + b*b
    aak2=a*a*k*2
    ddk=d*d*k
    num = (ksq + aak2 + a4b2) * (1 + ddk)
    den = a4b2 + aak2 + ddk*a4b2
    return l * num / den

def pade_31_num_jac(k, ksq, params, step=1e-2, tiny=1e-4):
    """ 
    Pade [3,1]  
    """
    f = pade_31(k, ksq, params)
    l = params[0]
    a = params[1]
    b = params[2]
    d = params[3]
    h = l*step + tiny
    dfdl = (pade_31(k, ksq, [l+h,a,b,d]) - pade_31(k, ksq, [l-h,a,b,d]))/(2*h)
    h = a*step + tiny
    dfda = (pade_31(k, ksq, [l,a+h,b,d]) - pade_31(k, ksq, [l,a-h,b,d]))/(2*h)
    h = b*step + tiny
    dfdb = (pade_31(k, ksq, [l,a,b+h,d]) - pade_31(k, ksq, [l,a,b-h,d]))/(2*h)
    h = d*step + tiny
    dfdd = (pade_31(k, ksq, [l,a,b,d+h]) - pade_31(k, ksq, [l,a,b,d-h]))/(2*h)
    return f, np.array([dfdl, dfda, dfdb, dfdd])

In [6]:
def chi2_gen_j(params, ks, k2s, lbs, pade):
    """
    d_chi2 = 1/N sum_i (f(kappa) - lambda) * df
    """
    n_kappa = len(ks)
    n_param = len(params)
    fs, dfs = pade(ks, k2s, params)
    diffs = fs - lbs
    chi2 = np.sum(np.square(diffs)) / n_kappa
    dchi2 = np.zeros(n_param)
    for ip in range(n_param):
        dchi2[ip] = 2./n_kappa * np.sum(diffs*dfs[ip])
    return chi2, dchi2

def pade_31j(k, ksq, params):
    """
    see RAC-31_derivatives notebook
    """
    l = params[0]
    a = params[1]
    b = params[2]
    d = params[3]
    a2 = a*a
    b2 = b*b
    d2 = d*d
    a4b2 = a2*a2 + b2
    aak2 = a2*k*2
    ddk = d2*k
    fr1 = (ksq + aak2 + a4b2)
    fr2 = (1 + ddk)
    den = a4b2 + aak2 + ddk*a4b2
    dl = fr1*fr2/den
    f = l*dl
    da = -4*a*ksq*l * fr2 * (a2*a2*d2 + a2*fr2 - b2*d2 + k) / den**2
    db = -2*b*ksq*l * fr2 * (2*a2*d2 + fr2) / den**2
    dd = 4*a2*d*ksq*l * fr1/den**2
    return f, np.array([dl, da, db, dd])

In [7]:
E0, G0 = 2.00, 2/7
p0s=[1.1] + racx.guess(E0, G0) + [0.5]

In [8]:
#
#  the derivative of [3,1] works
#
#pade_31(ks, k2s, p0s)
f1s, df1s = pade_31_num_jac(ks[88], k2s[88], p0s, step=1e-2, tiny=1e-4)
print("num grad:", df1s)

f2s, df2s = pade_31j(ks[88], k2s[88], p0s)
print("ana grad:", df2s)

#print("abs error", df1s-df2s)
#print("rel error", (df1s-df2s)/df2s )
#np.sum(fs*dfs[0])

num grad: [ 7.49507394 -4.98188982 -9.18993079  1.41923287]
ana grad: [ 7.49507394 -4.98196024 -9.18854002  1.41924818]


In [9]:
#
#  this is the reference when called with jac=False so that BFGS calls its own jac
#
chi2_gen(p0s, ks, k2s, ls, pade_31)

52.415954742841656

In [10]:
#
#  pade-independent testing
#
chi2_gen_num_jac(p0s, ks, k2s, ls, pade_31)

(52.415954742841656,
 array([-66.9006039 ,  36.63069754,  76.70161933, -10.38224543]))

In [11]:
#
#  analytical gradients for [3,1]
#
chi2_gen_j(p0s, ks, k2s, ls, pade_31j)

(52.415954742841656,
 array([-66.9006039 ,  36.63070077,  76.70160859, -10.38224678]))

In [12]:
E0, G0 = 2.00, 2/7
p0s=[1.1] + racx.guess(E0, G0) + [0.5]

In [13]:
# either bounds or constraints can be used to force positive parameters

bnds = ((0, None), (0, None), (0, None), (0, None))

cons = ({'type': 'ineq', 'fun': lambda x: x[0] },
        {'type': 'ineq', 'fun': lambda x: x[1] },
        {'type': 'ineq', 'fun': lambda x: x[2] },
        {'type': 'ineq', 'fun': lambda x: x[3] })

In [15]:
#
#  start with BFGS (the golden standard)
#

#
#  test results of other minimizers wrt BFGS
#
#p0s=res.x

print("Start parameters:",p0s)
print('Internal numerical gradient')
res = minimize(chi2_gen, p0s, args=(ks, k2s, ls, pade_31), 
               method='BFGS', options={'gtol':1e-7})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev, res.njev])
print("chi=%e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d, nG=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))
print('analytic gradient')
res = minimize(chi2_gen_j, p0s, args=(ks, k2s, ls, pade_31j), 
               method='BFGS', jac=True, options={'gtol':1e-7})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev, res.njev])
print("chi=%e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d, nG=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Start parameters: [1.1, 0.22466750413864453, 1.4151140501139234, 0.5]
Internal numerical gradient
chi=3.015960e-06,  l0=2.402191,  a=-0.271649,  b=1.281240,  d=-0.455012,  Iter=185, nE=1806, nG=299
Er=1.636130,  Gamma=0.378188
analytic gradient
chi=3.015598e-06,  l0=2.402286,  a=-0.271398,  b=1.281319,  d=-0.454389,  Iter=189, nE=233, nG=233
Er=1.636354,  Gamma=0.377511


In [16]:
print('Nelder-Mead')
res = minimize(chi2_gen, p0s, args=(ks, k2s, ls, pade_31), 
               method='Nelder-Mead', options={'fatol':1e-8})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Nelder-Mead
chi=9.616e-05,  l0=2.440436,  a=0.169568,  b=1.321514,  d=0.023226,  Iter=257, nE=441
Er=1.745572,  Gamma=0.151992


In [17]:
print('Conjugate gradient')
res = minimize(chi2_gen_j, p0s, args=(ks, k2s, ls, pade_31j), 
               method='CG', jac=True, options={'gtol':1e-7})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev, res.njev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d, nG=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Conjugate gradient
chi=3.016e-06,  l0=2.402289,  a=0.271386,  b=1.281322,  d=0.454359,  Iter=168, nE=418, nG=418
Er=1.636362,  Gamma=0.377480


In [18]:
alg='L-BFGS-B'
print(alg, 'with bounds')
res = minimize(chi2_gen_j, p0s, args=(ks, k2s, ls, pade_31j), 
               method=alg, jac=True, bounds=bnds, options={'gtol':1e-7})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

L-BFGS-B with bounds
chi=3.016e-06,  l0=2.402240,  a=0.271492,  b=1.281284,  d=0.454616,  Iter=65, nE=82
Er=1.636255,  Gamma=0.377762


In [19]:
print('TNC with bounds')
res = minimize(chi2_gen_j, p0s, args=(ks, k2s, ls, pade_31j), 
               method='TNC', jac=True, bounds=bnds, options={'gtol':1e-7})
print(res.fun)
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

TNC with bounds
9.686276402626803e-05
chi=9.686e-05,  l0=2.440407,  a=0.169329,  b=1.321558,  d=0.000000,  Iter=20, nE=83
Er=1.745694,  Gamma=0.151568


In [29]:
print('COBYLA with constraints')
res = minimize(chi2_gen, p0s, args=(ks, k2s, ls, pade_31), 
               method='COBYLA', tol=1e-7, constraints=cons, options={'maxiter':100000})
out = tuple([res.fun] + list(res.x) + [res.nfev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  nE=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

COBYLA with constraints
chi=2.838e-04,  l0=2.309875,  a=0.452694,  b=1.356868,  d=1.639606,  nE=100000
Er=1.799094,  Gamma=1.112260


In [21]:
#
#  Sequential least squares programming (allows constraints) 
#
print('Internal numerical gradient')
res = minimize(chi2_gen, p0s, args=(ks, k2s, ls, pade_31), 
               method='SLSQP', constraints=cons, options={'ftol':1e-7})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev, res.njev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d, nG=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))
print('analytic gradient')
res = minimize(chi2_gen_j, p0s, args=(ks, k2s, ls, pade_31j), 
               method='SLSQP', jac=True, constraints=cons, options={'ftol':1e-7})
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev, res.njev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d, nG=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Internal numerical gradient
chi=1.562e-04,  l0=2.338160,  a=0.444701,  b=1.309575,  d=1.090279,  Iter=24, nE=152, nG=24
Er=1.675878,  Gamma=1.035923
analytic gradient
chi=1.562e-04,  l0=2.338176,  a=0.444694,  b=1.309589,  d=1.090312,  Iter=24, nE=32, nG=24
Er=1.675917,  Gamma=1.035901


In [31]:
print('Trust Constraint')
res = minimize(chi2_gen_j, p0s, args=(ks, k2s, ls, pade_31j), 
               method='trust-constr', constraints=cons, jac=True)
out = tuple([res.fun] + list(res.x) + [res.nit, res.nfev, res.njev])
print("chi=%.3e,  l0=%f,  a=%f,  b=%f,  d=%f,  Iter=%d, nE=%d, nG=%d" % out)
print("Er=%f,  Gamma=%f" % racx.res_ene(res.x[1], res.x[2]))

Trust Constraint


/home/thomas/psi4conda/lib/python3.6/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


chi=1.700e-04,  l0=2.336261,  a=0.447424,  b=1.316658,  d=1.142091,  Iter=1001, nE=998, nG=998
Er=1.693514,  Gamma=1.054319


In [ ]:
lw = np.zeros(4)
up = np.full((4), np.inf)

args=(ks, k2s, ls, pade_31)
local_min={'method'='BFGS'}

ret = dual_annealing(func, args=args, bounds=list(zip(lw, up)), seed=1234)

In [ ]:
#
#  Try to make least_squares work
#
def chi_21(ps, ks, ls):
    """ Pade [2.1]
    l = params[0]
    a = params[1]
    b = params[2]
    a4b2=a*a*a*a + b*b
    aak2=a*a*k*2
    return l*(k*k + aak2 + a4b2) / (a4b2 + aak2)
    """
    return ps[0]*(np.square(ks) + 2*ps[1]**2*ks + ps[1]**4 + ps[2]**2) / (2*ps[1]**2*ks + ps[1]**4 + ps[2]**2) - ls

bnds=(0, np.inf)
p0s=[2.4407012,  0.16927274,  1.321711]
p0s=[1] + racx.guess(E0, G0)
res_lsq = least_squares(chi_21, p0s, args=(ks, ls), bounds=bnds)

In [ ]:
print(p0s)
print(res_lsq.x)
racx.res_ene(res_lsq.x[1], res_lsq.x[2])

In [ ]:
res_lsq.optimality

In [ ]:
res_lsq.cost